## Setup

Requires Python 3.10 and Poetry 1.6. Run setup commands to begin:
```
!poetry install
!poetry run jupyter lab
```

In [1]:
import os
import copy
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
display_plots = False

Load data

In [2]:
def get_raw_data():
    return {
        "A": {
            "test_est": pd.read_parquet(os.path.join("data", "A", "X_test_estimated.parquet")),
            "train_est": pd.read_parquet(os.path.join("data", "A", "X_train_estimated.parquet")),
            "train_obs": pd.read_parquet(os.path.join("data", "A", "X_train_observed.parquet")),
            "train_tar": pd.read_parquet(os.path.join("data", "A", "train_targets.parquet")),
        },
        "B": {
            "test_est": pd.read_parquet(os.path.join("data", "B", "X_test_estimated.parquet")),
            "train_est": pd.read_parquet(os.path.join("data", "B", "X_train_estimated.parquet")),
            "train_obs": pd.read_parquet(os.path.join("data", "B", "X_train_observed.parquet")),
            "train_tar": pd.read_parquet(os.path.join("data", "B", "train_targets.parquet")),
        },
        "C": {
            "test_est": pd.read_parquet(os.path.join("data", "C", "X_test_estimated.parquet")),
            "train_est": pd.read_parquet(os.path.join("data", "C", "X_train_estimated.parquet")),
            "train_obs": pd.read_parquet(os.path.join("data", "C", "X_train_observed.parquet")),
            "train_tar": pd.read_parquet(os.path.join("data", "C", "train_targets.parquet")),
        },
    }
data = get_raw_data()

View raw data

In [3]:
for loc in data:
    for type in data[loc]:
        display(loc, type, data[loc][type])

'A'

'test_est'

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-04-30 07:00:04,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,0.0,0.0,0.0,0.0,80.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,1000.700012,1007.000000,0.0,0.0,80.300003,1013.299988,NaN,0.0,0.0,0.0,0.0,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-04-30 07:00:04,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,0.0,0.0,0.0,0.0,77.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.700012,0.0,0.0,1000.599976,1006.900024,0.0,0.0,80.300003,1013.200012,NaN,0.0,0.0,0.0,0.0,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-04-30 07:00:04,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,0.0,0.0,0.0,0.0,73.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.599976,0.0,0.0,1000.500000,1006.700012,0.0,0.0,80.300003,1013.000000,NaN,0.0,0.0,0.0,0.0,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-04-30 07:00:04,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,0.0,0.0,0.0,0.0,69.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.500000,0.0,0.0,1000.400024,1006.599976,0.0,0.0,80.199997,1012.900024,NaN,0.0,0.0,0.0,0.0,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-04-30 07:00:04,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,0.0,0.0,0.0,0.0,64.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.299988,0.0,0.0,1000.200012,1006.500000,0.0,0.0,80.199997,1012.799988,NaN,0.0,0.0,0.0,0.0,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-07-02 07:00:31,2023-07-03 22:45:00,9.1,1.207,2090.600098,0.0,0.0,1629.400024,0.0,282.700012,0.0,0.0,0.0,0.0,99.300003,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.299988,0.0,0.0,980.799988,986.700012,0.0,0.0,82.199997,992.599976,NaN,0.0,0.0,-0.0,0.0,351.855988,-3.282,0.1,285.399994,99.400002,39931.601562,2.1,1.7,-1.3,0.0
2876,2023-07-02 07:00:31,2023-07-03 23:00:00,9.1,1.207,2015.800049,0.0,0.0,1537.300049,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.400024,0.0,0.0,980.900024,986.799988,0.0,0.0,82.500000,992.700012,NaN,0.0,0.0,-0.0,0.0,355.317993,-3.469,0.1,285.299988,100.000000,39574.300781,2.0,1.6,-1.3,0.0
2877,2023-07-02 07:00:31,2023-07-03 23:15:00,9.1,1.208,2015.699951,0.0,0.0,1471.400024,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.500000,0.0,0.0,981.000000,986.900024,0.0,0.0,82.800003,992.799988,NaN,0.0,0.0,-0.0,0.0,358.785004,-3.556,0.1,285.299988,100.000000,39627.199219,2.0,1.6,-1.2,0.0
2878,2023-07-02 07:00:31,2023-07-03 23:30:00,9.0,1.208,2015.599976,0.0,0.0,1405.500000,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.599976,0.0,0.0,981.000000,986.900024,0.0,0.0,83.199997,992.900024,NaN,0.0,0.0,-0.0,0.0,2.253000,-3.541,0.1,285.200012,100.000000,39691.800781,2.0,1.6,-1.1,0.0


'A'

'train_est'

...

'train_obs'

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2019-01-01 00:00:00,5.6,1.240,1153.599976,0.0,0.0,1153.599976,1.0,275.200012,0.0,0.0,0.0,0.0,63.599998,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,987.299988,0.00,0.0,972.599976,978.599976,0.0,0.0,90.199997,984.700012,NaN,0.0,0.0,-0.0,0.0,12.989000,-49.178001,0.0,278.500000,68.500000,18220.400391,2.5,1.1,2.2,-0.0
1,2019-01-01 00:15:00,5.5,1.241,1137.900024,0.0,0.0,1137.900024,1.0,275.100006,0.0,0.0,0.0,0.0,74.900002,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,987.099976,0.06,1.0,972.299988,978.299988,0.2,0.0,90.000000,984.400024,NaN,0.0,0.0,-0.0,0.0,18.214001,-48.724998,0.0,278.399994,80.800003,17083.699219,2.8,1.1,2.5,-0.0
2,2019-01-01 00:30:00,5.4,1.241,1122.199951,0.0,0.0,1122.199951,1.0,274.899994,0.0,0.0,0.0,0.0,84.300003,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.799988,0.06,1.0,972.000000,978.000000,0.3,0.0,89.900002,984.099976,NaN,0.0,0.0,-0.0,0.1,23.351000,-48.127998,0.0,278.299988,89.900002,15940.900391,3.0,1.1,2.8,-0.0
3,2019-01-01 00:45:00,5.4,1.241,1106.400024,0.0,0.0,1106.400024,1.0,274.700012,0.0,0.0,0.0,0.0,92.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.599976,0.06,1.0,971.700012,977.799988,0.3,0.0,89.699997,983.799988,NaN,0.0,0.0,-0.0,0.1,28.382000,-47.393002,0.0,278.200012,96.099998,14792.000000,3.3,1.2,3.0,-0.0
4,2019-01-01 01:00:00,5.3,1.241,1090.599976,0.0,0.0,1090.599976,1.0,274.500000,0.0,0.0,0.0,0.0,98.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.400024,0.06,1.0,971.400024,977.500000,0.4,0.0,89.599998,983.599976,NaN,0.0,0.0,-0.0,0.1,33.292999,-46.529999,0.0,278.100006,99.400002,13637.299805,3.5,1.2,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116820,2022-05-01 21:00:00,4.9,1.257,2652.000000,0.0,0.0,1536.500000,0.0,273.600006,0.0,0.0,0.0,0.0,95.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.799988,0.00,0.0,999.299988,1005.400024,0.0,0.0,62.599998,1011.599976,NaN,0.0,0.0,-0.0,0.0,327.303009,-7.367000,0.0,279.299988,100.000000,38312.000000,4.7,3.0,3.6,-0.0
116821,2022-05-01 21:15:00,4.9,1.256,2375.300049,0.0,0.0,1484.500000,0.0,273.600006,0.0,0.0,0.0,0.0,97.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.500000,0.00,0.0,999.000000,1005.099976,0.0,0.0,62.799999,1011.299988,NaN,0.0,0.0,-0.0,0.0,330.863007,-8.232000,0.1,279.299988,100.000000,38344.199219,4.8,3.2,3.6,-0.0
116822,2022-05-01 21:30:00,4.9,1.256,2098.699951,0.0,0.0,1432.500000,0.0,273.500000,0.0,0.0,0.0,0.0,98.500000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.200012,0.00,0.0,998.599976,1004.799988,0.0,0.0,63.000000,1010.900024,NaN,0.0,0.0,-0.0,0.0,334.451996,-9.006000,0.1,279.200012,100.000000,38402.601562,4.9,3.3,3.6,-0.0
116823,2022-05-01 21:45:00,4.8,1.255,1822.000000,0.0,0.0,1380.400024,0.0,273.399994,0.0,0.0,0.0,0.0,99.500000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.00,0.0,998.299988,1004.400024,0.0,0.0,63.200001,1010.599976,NaN,0.0,0.0,-0.0,0.0,338.067993,-9.683000,0.1,279.200012,100.000000,38478.800781,5.0,3.5,3.6,-0.0


'C'

'train_tar'

,time,pv_measurement
0,2018-12-31 23:00:00,NaN
1,2019-01-01 00:00:00,NaN
2,2019-01-01 01:00:00,NaN
3,2019-01-01 02:00:00,NaN
4,2019-01-01 03:00:00,NaN
...,...,...
32150,2023-04-30 19:00:00,50.96
32151,2023-04-30 20:00:00,2.94
32152,2023-04-30 21:00:00,0.00
32153,2023-04-30 22:00:00,-0.00


Find the number of unique values for each type

In [4]:
pd.concat([data[loc][type] for loc in data for type in data[loc]]).nunique()

date_calc                            214
date_forecast                     153805
absolute_humidity_2m:gm3             183
air_density_2m:kgm3                  301
ceiling_height_agl:m               78367
clear_sky_energy_1h:J             217030
clear_sky_rad:W                     8371
cloud_base_agl:m                   61031
dew_or_rime:idx                        3
dew_point_2m:K                       473
diffuse_rad:W                       3204
diffuse_rad_1h:J                  215787
direct_rad:W                        6590
direct_rad_1h:J                   180598
effective_cloud_cover:p             1001
elevation:m                            3
fresh_snow_12h:cm                    184
fresh_snow_1h:cm                      58
fresh_snow_24h:cm                    205
fresh_snow_3h:cm                     104
fresh_snow_6h:cm                     145
is_day:idx                             2
is_in_shadow:idx                       2
msl_pressure:hPa                     981
precip_5min:mm  

Plot target value

In [5]:
if display_plots:
    for loc in data:
        obs_start = data[loc]["train_obs"]["date_forecast"].min()
        obs_end = data[loc]["train_obs"]["date_forecast"].max()
        est_start = data[loc]["train_est"]["date_forecast"].min()
        est_end = data[loc]["train_est"]["date_forecast"].max()
        
        ax = data[loc]["train_tar"].set_index("time").plot(lw=0.1)
        ax.axvline(obs_start, linestyle='--')
        ax.axvline(obs_end, linestyle='--')
        ax.axvline(est_start, linestyle='--')
        ax.axvline(est_end, linestyle='--')
        ax.set_title("pv_measurement")
        plt.show()

## Transform data

Transform the raw data to be on the X y form

In [6]:
def transform_data(data):
    # Create empty list of location train data and location test data
    Xs = {"og" : {loc : {} for loc in data}}
    ys = {"og" : {loc : {} for loc in data}}
    
    for loc in data:
        # Copy data
        data_obs = data[loc]["train_obs"].copy()
        data_est = data[loc]["train_est"].copy()
        data_test = data[loc]["test_est"].copy()
        data_tar = data[loc]["train_tar"].copy()
        
        # Add a column with the difference between date_forecast and date_calc in seconds and 
        data_obs["forecast_seconds"] = np.zeros(data_obs.shape[0], dtype=np.float32)
        data_est["forecast_seconds"] = (data_est["date_forecast"] - data_est["date_calc"]).apply(lambda x: x.total_seconds()).astype('float32')
        data_test["forecast_seconds"] = (data_test["date_forecast"] - data_test["date_calc"]).apply(lambda x: x.total_seconds()).astype('float32')
        
        # Remove unneeded columns
        del data_est["date_calc"]
        del data_test["date_calc"]
        del data_obs["snow_density:kgm3"]
        del data_est["snow_density:kgm3"]
        del data_test["snow_density:kgm3"]

        # Find start and end date for obvsersed and estimated data
        obs_end = data_obs["date_forecast"].max()
        est_start = data_est["date_forecast"].min()
        treshold_date = obs_end + (est_start - obs_end) / 2
        
        # Outer join target data with estimated and observed data
        Xy_obs = pd.merge(
            data_tar[data_tar["time"] <= treshold_date],
            data_obs.rename(columns={"date_forecast": "time"}),
            on="time",
            how="outer"
        ).set_index("time").sort_index()
        Xy_est = pd.merge(
            data_tar[data_tar["time"] > treshold_date],
            data_est.rename(columns={"date_forecast": "time"}),
            on="time",
            how="outer"
        ).set_index("time").sort_index()

        # Assign Xs
        Xs["og"][loc]["obs"] = Xy_obs.drop("pv_measurement", axis=1)
        Xs["og"][loc]["est"] = Xy_est.drop("pv_measurement", axis=1)
        Xs["og"][loc]["test"] = data_test.rename(columns={"date_forecast": "time"}).set_index("time").sort_index()

        # Assign ys
        ys["og"][loc]["obs"] = Xy_obs["pv_measurement"]
        ys["og"][loc]["est"] = Xy_est["pv_measurement"]
        
    return Xs, ys

Xs, ys = transform_data(data)

Check that they have the same columns

In [7]:
def has_same_columns(df_lst):
    return all(set(df.columns) == set(df_lst[0].columns) for df in df_lst)

def are_all_equal(lst):
    return all(x == lst[0] for x in lst)

Xs_cols = Xs["og"]["A"]["obs"].columns.copy()
ys_col = ys["og"]["A"]["obs"].name
display(Xs["og"]["A"]["obs"].dtypes)

assert has_same_columns([Xs["og"][loc][type] for loc in Xs["og"] for type in Xs["og"][loc]]) 
assert are_all_equal([ys["og"][loc][type].name for loc in ys["og"] for type in ys["og"][loc]])

absolute_humidity_2m:gm3          float32
air_density_2m:kgm3               float32
ceiling_height_agl:m              float32
clear_sky_energy_1h:J             float32
clear_sky_rad:W                   float32
cloud_base_agl:m                  float32
dew_or_rime:idx                   float32
dew_point_2m:K                    float32
diffuse_rad:W                     float32
diffuse_rad_1h:J                  float32
direct_rad:W                      float32
direct_rad_1h:J                   float32
effective_cloud_cover:p           float32
elevation:m                       float32
fresh_snow_12h:cm                 float32
fresh_snow_1h:cm                  float32
fresh_snow_24h:cm                 float32
fresh_snow_3h:cm                  float32
fresh_snow_6h:cm                  float32
is_day:idx                        float32
is_in_shadow:idx                  float32
msl_pressure:hPa                  float32
precip_5min:mm                    float32
precip_type_5min:idx              

See stats about the data

In [8]:
pd.concat([Xs["og"][loc][type] for loc in Xs["og"] for type in Xs["og"][loc]]).describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_seconds
count,413791.000000,413791.000000,338248.000000,4.137910e+05,413791.000000,381929.000000,413791.000000,413791.000000,413791.000000,4.137910e+05,413791.000000,4.137910e+05,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000
mean,6.022480,1.255468,2796.203857,5.340971e+05,148.360306,1717.518188,0.002364,275.193665,40.693062,1.464950e+05,51.762341,1.863446e+05,67.066154,12.303769,0.139911,0.011645,0.276628,0.035001,0.069955,0.490383,0.559183,1009.625610,0.005670,0.084200,995.809204,1001.939941,0.851963,0.009438,73.572304,1008.098511,0.322603,0.000027,0.000382,0.088049,179.594162,-0.685629,0.056825,279.437927,73.616234,33047.644531,3.017195,0.677667,0.670873,0.000048,15049.955078
std,2.767936,0.037358,2488.790527,8.383486e+05,233.465897,1775.247437,0.253579,7.009235,61.931904,2.201175e+05,115.628448,4.105117e+05,34.583103,8.251439,0.842233,0.118890,1.340529,0.297705,0.511651,0.499908,0.496486,12.998659,0.032662,0.386233,12.919853,12.974976,5.750685,0.042001,14.360833,13.032352,1.823088,0.005156,0.005010,0.238048,103.120628,24.189608,0.108665,6.691157,34.396370,17905.146484,1.749300,2.778564,1.879975,0.006740,37329.292969
min,0.500000,1.139000,27.600000,0.000000e+00,0.000000,27.299999,-1.000000,247.300003,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,944.299988,0.000000,0.000000,929.799988,935.599976,0.000000,0.000000,18.700001,941.400024,0.000000,0.000000,-0.000000,0.000000,0.002000,-50.042000,0.000000,257.899994,0.000000,130.600006,0.000000,-7.300000,-9.300000,-0.100000,0.000000
25%,4.000000,1.230000,1048.199951,0.000000e+00,0.000000,596.700012,0.000000,270.600006,0.000000,0.000000e+00,0.000000,0.000000e+00,41.599998,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.500000,0.000000,0.000000,987.799988,993.900024,0.000000,0.000000,64.099998,1000.000000,0.000000,0.000000,0.000000,0.000000,89.743500,-17.895000,0.000000,274.799988,52.400002,16864.900391,1.700000,-1.300000,-0.600000,-0.000000,0.000000
50%,5.400000,1.255000,1821.800049,9.416100e+03,0.000000,1158.300049,0.000000,275.000000,0.000000,1.198950e+04,0.000000,0.000000e+00,80.400002,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1010.500000,0.000000,0.000000,996.700012,1002.799988,0.000000,0.000000,76.000000,1009.000000,0.000000,0.000000,0.000000,0.000000,179.462997,-0.563000,0.000000,278.700012,93.699997,37022.199219,2.700000,0.300000,0.700000,0.000000,0.000000
75%,7.900000,1.279000,3795.424805,8.397916e+05,233.699997,2067.500000,0.000000,280.600006,68.300003,2.453228e+05,30.500000,1.181203e+05,98.900002,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1018.599976,0.000000,0.000000,1004.900024,1011.000000,0.000000,0.000000,85.099998,1017.20001

Plot timeperiod of test dates

In [9]:
extract_dates = {loc: Xs["og"][loc]["test"].index.copy() for loc in Xs["og"]}

if display_plots:
    for height, loc in zip([3,2,1], extract_dates):
        plt.plot(extract_dates[loc], [height] * len(extract_dates[loc]), marker='o', markersize=1, linestyle='')
        
    display(extract_dates[loc].to_series().diff().value_counts())

    plt.gca().axes.get_yaxis().set_visible(False)    
    plt.ylim(0, 4)
    plt.legend([loc for loc in extract_dates])
    plt.xticks(rotation=45)
    plt.show()

def are_all_equal_series(series_lst):
    return all((series == series_lst[0]).all() for series in series_lst)

assert are_all_equal_series([extract_dates[loc] for loc in extract_dates])
extract_dates = extract_dates["A"]

Display ys

In [10]:
if display_plots:
    for loc in ys["og"]:
        for type in ys["og"][loc]:
            ax = ys["og"][loc][type].dropna().plot(lw=0.1)
    
    ax.set_title(ys_col)
    legend = ax.legend([loc + " " + type for loc in ys["og"] for type in ys["og"][loc]])
    for line in legend.get_lines():
        line.set_linewidth(1)
    plt.show()
    
    for loc in ys["og"]:
        for type in ys["og"][loc]:
            ax = ys["og"][loc][type].dropna().plot(lw=0.1)
    
        ax.set_title(loc)
        plt.show()

Display Xs only with nan values

In [11]:
Xs_cols_with_nan = Xs_cols[pd.concat([Xs["og"][loc][type].dropna(how='all') for loc in Xs["og"] for type in Xs["og"][loc]]).isna().any()]
display(Xs_cols_with_nan)

if display_plots:
    for col in Xs_cols_with_nan:
        for loc in Xs["og"]:
            for type in Xs["og"][loc]:
                ax = Xs["og"][loc][type][col].dropna().plot(lw=0.1)
    
        ax.set_title(col)
        legend = ax.legend([loc + " " + type for loc in Xs["og"] for type in Xs["og"][loc]])
        for line in legend.get_lines():
            line.set_linewidth(1)
        plt.show()

Index(['ceiling_height_agl:m', 'cloud_base_agl:m'], dtype='object')

Plot histogram of the data

In [12]:
if display_plots:
    pd.concat([Xs["og"][loc][type] for loc in Xs["og"] for type in Xs["og"][loc]]).hist(figsize=(20,20))
    plt.show()

Plot correlation matrix of the data

In [13]:
if display_plots:
    display(list(zip(range(len(Xs_cols)), Xs_cols)))
    plt.matshow(pd.concat([Xs["og"][loc][type] for loc in Xs["og"] for type in Xs["og"][loc]]).corr(), cmap="coolwarm", vmin=-1, vmax=1)
    plt.show()

## Gap detection

Plot timeperiod of test dates

In [14]:
def gap_detection_in_dates(dates, min_gap_time):
    diff = dates.to_series().diff()
    return list(zip(diff[diff.shift(-1) > min_gap_time].index, diff[diff > min_gap_time].index))

def get_non_nan_dates(X):
    return X.index[:1].union(X.dropna(how='all').index.union(X.index[-1:]))

def date_gaps_detection(X, min_gap_time):
    return gap_detection_in_dates(get_non_nan_dates(X), min_gap_time)

min_gap_time = pd.Timedelta(days=1)

if display_plots:
    gaps = gap_detection_in_dates(extract_dates, min_gap_time)
    
    plt.plot(extract_dates, [1] * len(extract_dates), marker='o', markersize=1, linestyle='')

    ax = plt.gca()
    for start_date, end_date in gaps:
        ax.axvline(start_date, linestyle='--', color="green")
        ax.axvline(end_date, linestyle='--', color="red")
    ax.axes.get_yaxis().set_visible(False)
    plt.xticks(rotation=45)
    plt.show()

In [15]:
def value_gaps_detection(x, min_gap_time, exclude_vals={}, time_padding=pd.Timedelta(seconds=1)):    
    start_date = x.index[0] - time_padding
    start_val = x.iloc[0]

    gaps_date = []
    index = x.dropna().index
    
    for date in index:
        val = x[date]
        
        if(val != start_val or val in exclude_vals):
            if(date - start_date > min_gap_time):
                gaps_date.append((start_date, date))
    
            start_date = date
            start_val = val

    i = min(x.index.get_loc(index[-1]) + 1, len(x) - 1)
    if(x.index[i] - start_date > min_gap_time):
        gaps_date.append((start_date, x.index[i]))

    if(x.index[-1] - x.index[i-1] > min_gap_time):
        gaps_date.append((x.index[i-1], x.index[-1] + time_padding))

    return gaps_date

if display_plots:
    for loc in ys["og"]:
        for type in ys["og"][loc]:
            ax = ys["og"][loc][type].dropna().plot(lw=0.1)

            gaps = value_gaps_detection(ys["og"][loc][type], min_gap_time)
            for start_date, end_date in gaps:
                ax.axvline(start_date, linestyle='--', color="green")
                ax.axvline(end_date, linestyle='--', color="red")
            ax.set_title(loc)
        plt.show()

## Timeseries data preprocessing

Set constant gaps to NaN

In [16]:
def fill_constant_gaps_with_nan(x, min_gap_time):
    gaps = value_gaps_detection(x, min_gap_time)
    for (start_date, end_date) in gaps:        
        gap_index = x.index[(start_date < x.index) & (x.index < end_date)]
        x[gap_index] = np.nan

min_gap_time = pd.Timedelta(days=1)

for loc in ys["og"]:
    for type in ys["og"][loc]:
        fill_constant_gaps_with_nan(ys["og"][loc][type], min_gap_time)

Reindex to 15 min and 1 hour regular timeseries and combine all the a full timeseries

In [17]:
def regular_index(X, freq, start_padding=pd.Timedelta(0), end_padding=pd.Timedelta(0)):    
    return X.reindex(pd.date_range(start = X.index[0]-start_padding, end = X.index[-1]+end_padding, freq=freq))

gaps = date_gaps_detection(ys["og"]["C"]["obs"], min_gap_time)
Xs["og"]["C"]["obs"] = Xs["og"]["C"]["obs"][Xs["og"]["C"]["obs"].index >= gaps[0][1]]
ys["og"]["C"]["obs"] = ys["og"]["C"]["obs"][ys["og"]["C"]["obs"].index >= gaps[0][1]]

Xs["15min"] = {loc: {} for loc in Xs["og"]}
Xs["1hour"] = {loc: {} for loc in Xs["og"]}

ys["15min"] = {loc: {} for loc in ys["og"]}
ys["1hour"] = {loc: {} for loc in ys["og"]}

end_padding = pd.DateOffset(days=30)

for pre, freq in zip(["15min", "1hour"], ["15T", "H"]):
    for loc in Xs["og"]:
        for type in Xs["og"][loc]:
            Xs[pre][loc][type] = regular_index(Xs["og"][loc][type], freq)
        Xs[pre][loc]["full"] = regular_index(pd.concat(Xs["og"][loc].values()), freq, end_padding=end_padding)

    for loc in ys["og"]:
        for type in ys["og"][loc]:
            ys[pre][loc][type] = regular_index(ys["og"][loc][type], freq)
        ys[pre][loc]["full"] = regular_index(pd.concat(ys["og"][loc].values()), freq)
        
for loc in Xs["og"]:
    Xs["og"][loc]["full"] = pd.concat(Xs["og"][loc].values())
    ys["og"][loc]["full"] = pd.concat(ys["og"][loc].values())

Fill gaps with mean of neighbouring data 

In [18]:
def neighbour_gaps_filling(X, gaps, max_days, intersection_only=True, yearly=False):
    X_copy = X.copy()

    gaps_index = pd.DatetimeIndex([])
    for start_date, end_date in gaps:
        gaps_index = gaps_index.union(X.index[(start_date < X.index) & (X.index < end_date)])
    no_gaps_index = X.index.difference(gaps_index)
    
    for start_date, end_date in gaps:
        if yearly:
            min_days = 1  
        else:
            min_days = (end_date - start_date).days // 2 + 1

        target_index = X.index[(start_date < X.index) & (X.index < end_date)]
        accumelated_index = pd.DatetimeIndex([])

        for days in range(min_days, max_days+1):
            if yearly:
                date_offset = pd.DateOffset(years=days)
            else :
                date_offset = pd.DateOffset(days=days)
            
            past_target_index = no_gaps_index.intersection(target_index - date_offset) + date_offset
            future_target_index = no_gaps_index.intersection(target_index + date_offset) - date_offset
            
            intersection_target_index = past_target_index.intersection(future_target_index).difference(accumelated_index)
            
            if len(intersection_target_index) != 0:
                X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
                    X[X.index.isin(intersection_target_index - date_offset)].values,
                    X[X.index.isin(intersection_target_index + date_offset)].values
                ], axis=0)
                accumelated_index = accumelated_index.union(intersection_target_index)

            if not intersection_only:
                diff_past_target_index = past_target_index.difference(intersection_target_index)
                diff_future_target_index = future_target_index.difference(intersection_target_index)
                
                if len(diff_past_target_index) != 0:
                    X_copy[X.index.isin(diff_past_target_index)] = X[X.index.isin(diff_past_target_index - date_offset)].values
                    accumelated_index = accumelated_index.union(diff_past_target_index)
                    
                if len(diff_future_target_index) != 0:
                    X_copy[X.index.isin(diff_future_target_index)] = X[X.index.isin(diff_future_target_index + date_offset)].values
                    accumelated_index = accumelated_index.union(diff_future_target_index)

            if len(target_index.difference(accumelated_index)) == 0:
                break
    
    return X_copy

def while_fill_gaps(min_gap_time, max_days, X1, X2=None, intersection_only=True, yearly=False):
    last_gaps = []
    while True:
        gaps = date_gaps_detection(X1, min_gap_time)
        
        if gaps == last_gaps:
            break
        last_gaps = gaps
        
        if X2 is not None:
            X2 = neighbour_gaps_filling(X2, gaps, max_days, intersection_only, yearly)

        X1 = neighbour_gaps_filling(X1, gaps, max_days, intersection_only, yearly)

    if X2 is None:
        return X1
    else:
        return X1, X2

In [19]:
max_days = 3

for pre in ["15min", "1hour"]:
    for loc in Xs[pre]:
        for type in ys[pre][loc]:
            ys[pre][loc][type], Xs[pre][loc][type] = while_fill_gaps(min_gap_time, max_days, ys[pre][loc][type], Xs[pre][loc][type])
        
        for type in Xs[pre][loc]:
            Xs[pre][loc][type] = while_fill_gaps(min_gap_time, max_days, Xs[pre][loc][type])

if display_plots:
    for pre in ["15min", "1hour"]:
        for loc in ys[pre]:
            ax = ys[pre][loc]["full"].dropna().plot(lw=0.1)
            
            gaps = date_gaps_detection(ys[pre][loc]["full"], min_gap_time)
            for start_date, end_date in gaps:
                ax.axvline(start_date, linestyle='--', color="green")
                ax.axvline(end_date, linestyle='--', color="red")
            
            plt.title(pre + " " + loc + " full")
            plt.show()

    legends = []
    y = 0
    for pre in Xs:
        for loc in Xs[pre]:
            y = y + 1
            legends.append(pre + " " + loc + " " + type)
            dates = Xs[pre][loc]["test"].dropna(how='all').index
            plt.plot(dates, [y] * len(dates), marker='o', markersize=1, linestyle='')

    plt.gca().axes.get_yaxis().set_visible(False)
    plt.legend(legends)
    plt.xticks(rotation=45)
    plt.show()

/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354

In [20]:
max_years = 3

for pre in ["15min", "1hour"]:
    for loc in Xs[pre]:
        for type in ys[pre][loc]:
            fill_constant_gaps_with_nan(ys[pre][loc][type], min_gap_time)
            ys[pre][loc][type], Xs[pre][loc][type] = while_fill_gaps(
                min_gap_time, 
                max_years,
                ys[pre][loc][type], 
                Xs[pre][loc][type],
                intersection_only = False,
                yearly = True,
            )

        for type in Xs[pre][loc]:
            Xs[pre][loc][type] = while_fill_gaps(
                min_gap_time, 
                max_years,
                Xs[pre][loc][type], 
                intersection_only = False,
                yearly = True,
            )

if display_plots:
    for pre in ["15min", "1hour"]:
        for loc in ys[pre]:
            ax = ys[pre][loc]["full"].dropna().plot(lw=0.1)

            gaps = date_gaps_detection(ys[pre][loc]["full"], min_gap_time)
            for start_date, end_date in gaps:
                ax.axvline(start_date, linestyle='--', color="green")
                ax.axvline(end_date, linestyle='--', color="red")
            
            plt.title(pre + " " + loc + " full")
            plt.show()

/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([
/tmp/ipykernel_27060/791354386.py:30: RuntimeWarning: Mean of empty slice
  X_copy[X.index.isin(intersection_target_index)] = np.nanmean([


Time interpolate missing values

In [21]:
def time_interpolate(X):
    return X.interpolate(method='time', limit_direction="both")

for pre in ["15min", "1hour"]:
    for loc in Xs[pre]:
        for type in Xs[pre][loc]:
            Xs[pre][loc][type] = time_interpolate(Xs[pre][loc][type])

for pre in ["15min", "1hour"]:
    for loc in ys[pre]:
        for type in ys[pre][loc]:
            ys[pre][loc][type] = time_interpolate(ys[pre][loc][type])

Check for nan values

In [22]:
def nested_values(d, exclude_keys={}):
    values = []
    for key, value in d.items():
        if key not in exclude_keys:
            if isinstance(value, dict):
                values.extend(nested_values(value, exclude_keys))
            else:
                values.append(value)
    return values

display(pd.concat(nested_values(Xs, exclude_keys={"og"})).isna().sum())
display(pd.concat(nested_values(ys, exclude_keys={"og"})).isna().sum())

absolute_humidity_2m:gm3          0
air_density_2m:kgm3               0
ceiling_height_agl:m              0
clear_sky_energy_1h:J             0
clear_sky_rad:W                   0
cloud_base_agl:m                  0
dew_or_rime:idx                   0
dew_point_2m:K                    0
diffuse_rad:W                     0
diffuse_rad_1h:J                  0
direct_rad:W                      0
direct_rad_1h:J                   0
effective_cloud_cover:p           0
elevation:m                       0
fresh_snow_12h:cm                 0
fresh_snow_1h:cm                  0
fresh_snow_24h:cm                 0
fresh_snow_3h:cm                  0
fresh_snow_6h:cm                  0
is_day:idx                        0
is_in_shadow:idx                  0
msl_pressure:hPa                  0
precip_5min:mm                    0
precip_type_5min:idx              0
pressure_100m:hPa                 0
pressure_50m:hPa                  0
prob_rime:p                       0
rain_water:kgm2             

0

Check that the period is correct

In [23]:
for pre in ["15min", "1hour"]:
    for loc in Xs[pre]:
        for type in Xs[pre][loc]:
            display(
                "",
                "Xs " + pre + " " + loc + " " + type, 
                Xs[pre][loc][type].index[0], 
                Xs[pre][loc][type].index[-1], 
                Xs[pre][loc][type].index.to_series().diff().value_counts()
            )

    for loc in ys[pre]:
        for type in ys[pre][loc]:
            display(
                "",
                "ys " + pre + " " + loc + " " + type, 
                ys[pre][loc][type].index[0], 
                ys[pre][loc][type].index[-1], 
                ys[pre][loc][type].index.to_series().diff().value_counts()
            )

''

'Xs 15min A obs'

Timestamp('2019-06-02 22:00:00')

Timestamp('2022-10-21 01:00:00')

0 days 00:15:00    118668
Name: count, dtype: int64

...

''

'ys 1hour C full'

Timestamp('2019-09-04 08:00:00')

Timestamp('2023-04-30 23:00:00')

0 days 01:00:00    32031
Name: count, dtype: int64

## More data preprocessing

One hot encoding of categorial values in X

In [24]:
def one_hot_encoding(X):
    return pd.get_dummies(X)

Z-score normalizing of numeric values in X

In [25]:
class StandardScaler:
    def __init__(self, X):
        self.means = X.mean(numeric_only=True)
        self.stds = X.std(numeric_only=True)

    def normalize(self, X):
        indices = self.indices(X)
        X_copy = X.copy()
        X_copy[indices] = (X[indices] - self.means[indices]) / self.stds[indices]
        return X_copy

    def denormalize(self, X):
        indices = self.indices(X)
        X_copy = X.copy()
        X_copy[indices] = X[indices] * self.stds[indices] + self.means[indices]
        return X_copy

    def indices(self, X):
        return self.means.index.intersection(X.columns)

Polynomial expansion of numeric values in X to degree n
    
Example:
If [x y] = X then
- degree 1: [1 x y]
- degree 2: [1 x y xx xy yy]
- degree 3: [1 x y xx xy yy xxx xxy xyy yyy]
- ...

Note: Since multiplication is commutitative then 
    xy = yx, xxy = xyx = yxx, xyy = yxy = yyx
so we skip non-lexicographically sorted multiplications.

In [26]:
def is_sorted(lst):
    return all(lst[i] <= lst[i+1] for i in range(len(lst) - 1))

def polynomial_expansion(X, degree=2, with_constant_term=False):
    cols = X.select_dtypes(include=["number"]).columns
    X_num = X.select_dtypes(include=["number"]).to_numpy()
    (m,n) = X_num.shape

    xs = [X]
    if(with_constant_term):
        xs.append(pd.Series(np.ones(m), name='1'))

    for d in range(2, degree+1):
        for indices in filter(is_sorted, itertools.product(range(n), repeat=d)):
            x = np.ones(m)
            for j in indices:
                x *= X_num[:,j]
            xs.append(pd.Series(x, name="_x_".join([cols[i] for i in indices])))

    return pd.concat(xs, axis=1)

Test functions

In [27]:
X = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9],
    'D': ['a', 'b', 'c'],
})

scaler = StandardScaler(X)
X_norm = scaler.normalize(X)

print(scaler.denormalize(X_norm))
print(X_norm)
print(one_hot_encoding(X))
print(polynomial_expansion(X))

     A    B    C  D
0  1.0  4.0  7.0  a
1  2.0  5.0  8.0  b
2  3.0  6.0  9.0  c
     A    B    C  D
0 -1.0 -1.0 -1.0  a
1  0.0  0.0  0.0  b
2  1.0  1.0  1.0  c
   A  B  C    D_a    D_b    D_c
0  1  4  7   True  False  False
1  2  5  8  False   True  False
2  3  6  9  False  False   True
   A  B  C  D  A_x_A  A_x_B  A_x_C  B_x_B  B_x_C  C_x_C
0  1  4  7  a    1.0    4.0    7.0   16.0   28.0   49.0
1  2  5  8  b    4.0   10.0   16.0   25.0   40.0   64.0
2  3  6  9  c    9.0   18.0   27.0   36.0   54.0   81.0


Normalize Xs and ys to standard scalar with sklearn

In [28]:
from sklearn.preprocessing import StandardScaler

pers = ["og", "15min", "1hour"]

scaler_Xs = {}
scaler_ys = {}

for per in pers:
    per_norm = per + " norm"
    
    Xs[per_norm] = {}
    scaler_Xs[per_norm] = StandardScaler().fit(pd.concat(nested_values(Xs[per], exclude_keys={"full", "BC"})).to_numpy())
    for loc in Xs[per]:
        Xs[per_norm][loc] = {}
        for type in Xs[per][loc]:
            Xs[per_norm][loc][type] = pd.DataFrame(
                scaler_Xs[per_norm].transform(Xs[per][loc][type].to_numpy()), 
                columns = Xs[per][loc][type].columns, 
                index = Xs[per][loc][type].index,
            )
            display("Xs " + per + " " + loc + " " + type, Xs[per_norm][loc][type].mean().mean(), Xs[per_norm][loc][type].std().mean())

    ys[per_norm] = {}
    scaler_ys[per_norm] = StandardScaler().fit(pd.concat(nested_values(ys[per], exclude_keys={"full", "BC"})).to_numpy().reshape(-1, 1))
    for loc in ys[per]:
        ys[per_norm][loc] = {}
        for type in ys[per][loc]:
            ys[per_norm][loc][type] = pd.Series(
                scaler_ys[per_norm].transform(ys[per][loc][type].to_numpy().reshape(-1, 1)).flatten(), 
                name = ys[per][loc][type].name, 
                index = ys[per][loc][type].index,
            )
            display("ys " + per + " " + loc + " " + type, ys[per_norm][loc][type].mean().mean(), ys[per_norm][loc][type].std().mean())

'Xs og A obs'

-0.008288124

0.83411705

'Xs og A est'

-0.028572252

...

-0.33045395211468254

0.11890062254689324

'ys 1hour C full'

-0.25756245200862443

0.22893375158877705

Do PCA of Xs with sklearn

In [29]:
from sklearn.decomposition import PCA

variance_threshold = 0.99
pca_Xs = {}

for per in ["15min", "1hour"]:
    per_pca = per + " pca"
    per_norm = per + " norm"
    
    Xs[per_pca] = {}
    pca_Xs[per_pca] = PCA().fit(pd.concat(nested_values(Xs[per_norm], exclude_keys={"full", "BC"})).to_numpy())

    explained_variance = pca_Xs[per_pca].explained_variance_ratio_
    cumulative_variance = explained_variance.cumsum()
    n_components = (cumulative_variance < variance_threshold).sum() + 1
    display(per + ": " + str(n_components))
    
    if display_plots:
        plt.plot(cumulative_variance, color="black")
        plt.title(per)
        plt.xlabel("Number of Components")
        plt.ylabel("Cumulative Explained Variance")
        plt.axhline(0.99, linestyle='--', label='99% variance', color=(0.5, 0.0, 0.0))
        plt.axhline(0.95, linestyle='--', label='95% variance', color=(0.0, 0.5, 0.0))
        plt.axhline(0.90, linestyle='--', label='90% variance', color=(0.0, 0.0, 0.5))
        plt.legend()
        plt.show()
    
    for loc in Xs[per]:
        Xs[per_pca][loc] = {}
        for type in Xs[per][loc]:
            Xs[per_pca][loc][type] = pd.DataFrame(
                pca_Xs[per_pca].transform(Xs[per_norm][loc][type].to_numpy())[:,:n_components], 
                columns = [f'PC{i}' for i in range(n_components)], 
                index = Xs[per][loc][type].index,
            )

'15min: 32'

'1hour: 32'

Plot histogram of the data

In [30]:
if display_plots:
    for per in ["15min pca", "1hour pca"]:
        pd.concat(nested_values(Xs[per], exclude_keys={"full", "BC"})).hist(figsize=(20,20))
        plt.show()

Plot correlation matrix of the data

In [31]:
if display_plots:
    for per in ["15min pca", "1hour pca"]:
        plt.matshow(pd.concat(nested_values(Xs[per], exclude_keys={"full", "BC"})).corr(), cmap="coolwarm", vmin=-1, vmax=1)
        plt.show()

## TSfresh timeseries feature extraction

In [32]:
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

def tsfresh_format_og(Xs, pre, full):
    Xys_loc_train = []
    Xs_loc_test = []

    y_name = "pv_measurement"
    loc_name = "location"
    time_name = "time"
    
    for loc in ["A", "B", "C"]:
        if full:
            X_loc_full = time_interpolate(Xs[pre][loc]["full"])
            y_loc_train = ys[pre][loc]["full"].copy()
            X_loc_train = X_loc_full[X_loc_full.index <= y_loc_train.index[-1]].copy()
            X_loc_test = X_loc_full[X_loc_full.index > y_loc_train.index[-1]].copy()
        else:
            X_loc_train = time_interpolate(pd.concat([Xs[pre][loc]["obs"], Xs[pre][loc]["est"]]))
            y_loc_train = pd.concat([ys[pre][loc]["obs"], ys[pre][loc]["est"]])
            X_loc_test = time_interpolate(Xs[pre][loc]["test"].copy())
        
        Xy_loc_train = X_loc_train.join(y_loc_train, how='outer')
        Xy_loc_train = Xy_loc_train.reset_index().rename(columns={'index': time_name})
        Xy_loc_train[loc_name] = loc
        
        X_loc_test = X_loc_test.reset_index().rename(columns={'index': time_name})
        X_loc_test[loc_name] = loc

        Xys_loc_train.append(Xy_loc_train)
        Xs_loc_test.append(X_loc_test)

    Xy_train = pd.concat(Xys_loc_train).dropna(subset=[y_name])
    X_test = pd.concat(Xs_loc_test)

    X = pd.concat([Xy_train.drop(columns=[y_name]), X_test]).reset_index(drop=True).reset_index().rename(columns={'index': "id"})
    y = Xy_train[y_name].reset_index(drop=True)
    l = X[loc_name]
        
    return X.drop(columns=[loc_name]), y, l

do_tsfresh = False
if do_tsfresh:
    X, y, l = tsfresh_format_og(Xs, "og", full=False)
    
    display(X)
    display(y)
    display(l)
    
    extracted_features = extract_features(X, column_id="id", column_sort="time")
    
    impute(extracted_features)
    features_filtered = select_features(extracted_features.head(len(y)), y)
    
    X_tsfresh = extracted_features[features_filtered.columns]
    X_tsfresh["location"] = l
    
    Xs["og tsfresh"] = X_tsfresh
    ys["og tsfresh"] = y

## Darts setup

Convert to darts timeseries

In [33]:
from darts import TimeSeries

display_plots = True

loc_dummies = {
    "A": pd.DataFrame({'location_A': [1], 'location_B': [0], 'location_C': [0]}),
    "B": pd.DataFrame({'location_A': [0], 'location_B': [1], 'location_C': [0]}),
    "C": pd.DataFrame({'location_A': [0], 'location_B': [0], 'location_C': [1]}),
}

covs = {}
for pre in ["15min", "1hour", "15min norm", "1hour norm", "15min pca", "1hour pca"]:
    covs[pre] = {loc: {type: TimeSeries.from_dataframe(Xs[pre][loc][type], static_covariates=loc_dummies[loc]) for type in Xs[pre][loc] } for loc in Xs[pre]}

tars = {}
for pre in ["15min", "1hour", "15min norm", "1hour norm"]:
    tars[pre] = {loc: {type: TimeSeries.from_series(ys[pre][loc][type], static_covariates=loc_dummies[loc]) for type in ys[pre][loc] } for loc in ys[pre]}

locs = ["A", "B", "C"]

assert are_all_equal([ys["15min"][loc][type].index[-1] for loc in locs for type in ["est", "full"]])
assert are_all_equal([ys["1hour"][loc][type].index[-1] for loc in locs for type in ["est", "full"]])

n_15min = len(pd.date_range(start=ys["15min"]["A"]["est"].index[-1], end=extract_dates[-1], freq='15T'))
n_1hour = len(pd.date_range(start=ys["1hour"]["A"]["est"].index[-1], end=extract_dates[-1], freq='H'))

Darts filter

In [34]:
from darts.models import GaussianProcessFilter
from sklearn.gaussian_process.kernels import ExpSineSquared

do_gaussian_filter = False
if do_gaussian_filter:
    gaussian_filter = GaussianProcessFilter(kernel=ExpSineSquared())
    
    tars["1hour gaussian"] = {loc: gaussian_filter.filter(TimeSeries.from_series(regular_index(ys["og"][loc]["full"], "H"))) for loc in ys["og"]}
    
    if display_plots:
        for loc in tars["1hour gaussian"]:
            tars["1hour gaussian"][loc]["full"].plot()
            plt.show()

Save predictions to csv file

In [35]:
def extract_and_save(ys, file_name, tars_per, path="predictions", extract_dates=extract_dates, scaler_ys=scaler_ys):
    y = pd.concat([ys[loc].pd_series()[extract_dates[extract_dates.minute == 0]] for loc in ys])

    if tars_per in scaler_ys:
        y = pd.Series(
            scaler_ys[tars_per].inverse_transform(y.to_numpy().reshape(-1, 1)).flatten(),
            name = y.name, 
            index = y.index,
        )
    
    y[y < 0] = 0
    y.name = "prediction"
    y = y.reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
    y.to_csv(os.path.join(path, file_name), index=False)
    
    return y

In [36]:
def fjkld():
    return 10, 5

x = fjkld()[0]
display(x)

10

## CatBoost

Train and make predicitons

In [37]:
from darts.models import CatBoostModel

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_cat = {}

do_cat = True
if do_cat:
    cat = CatBoostModel(lags=5*24, lags_future_covariates=(24,24))
    cat.fit([tars[tars_per][loc][type] for loc in locs], future_covariates=[covs[covs_per][loc][type] for loc in locs])
    for loc in locs:
        ys_cat[loc] = cat.predict(n_1hour, tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=2280]="A": Cannot convert 'b'A'' to float

In [ ]:
if do_cat:
    y = extract_and_save(ys_cat, "cat_1hour.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()

## XGBoost

Train and make predicitons

In [ ]:
from darts.models import XGBModel

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_xgboost = {}

do_xgboost = True
if do_xgboost:
    xgb = XGBModel(lags=5*24, lags_future_covariates=(24,24))
    xgb.fit([tars[tars_per][loc][type] for loc in locs], future_covariates=[covs[covs_per][loc][type] for loc in locs])
    for loc in locs:
        ys_xgboost[loc] = xgb.predict(n_1hour, tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_xgboost:
    y = extract_and_save(ys_xgboost, "xbg_1hour.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()

## LightGBM

Train and make predicitons

In [ ]:
from darts.models import LightGBMModel

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_light = {}

do_light = True
if do_light:
    light = LightGBMModel(lags=5*24, lags_future_covariates=(24,24))
    light.fit([tars[tars_per][loc][type] for loc in locs], future_covariates=[covs[covs_per][loc][type] for loc in locs])
    for loc in locs:
        ys_light[loc] = light.predict(n_1hour, tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_light:
    y = extract_and_save(ys_light, "light_gbm_1hour.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()

## RandomForest

Train and make predicitons

In [ ]:
from darts.models import RandomForest

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_forest = {}

do_forest = True
if do_forest:
    for loc in tars[tars_per]:
        forest_model = RandomForest(lags=3*24, lags_future_covariates=(24,24))
        forest_model.fit(tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])
        ys_forest[loc] = forest_model.predict(n_1hour, future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_forest:
    y = extract_and_save(ys_forest, "random_forest_1hour.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()

## Prophet

Train and make predicitons

In [ ]:
from darts.models import Prophet

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_prophet = {}

do_prophet = True
if do_prophet:
    for loc in tars[tars_per]:
        prophet = Prophet()
        prophet.fit(tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])
        ys_prophet[loc] = prophet.predict(n_1hour, future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_prophet:
    y = extract_and_save(ys_prophet, "prophet_15min.csv", tars_per)
    if True:
        y["prediction"].plot(lw=0.5)
        plt.show()

## AutoARIMA

Train and make predicitons

In [ ]:
from darts.models import AutoARIMA

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_autoarima = {}

do_autoarima = True
if do_autoarima:
    for loc in tars[tars_per]:
        ari = AutoARIMA()
        ari.fit(tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])
        ys_autoarima[loc] = ari.predict(n_1hour, future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_autoarima:
    y = extract_and_save(ys_autoarima, "ari_15min.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()

## RNN model

Train and make predicitons

In [ ]:
from darts.models import RNNModel

tars_per = "1hour norm"
covs_per = "1hour pca"
type = "full"

ys_rnn = {}

do_rnn = False
if do_rnn:    
    rnn_model = RNNModel(
        model = 'LSTM',
        input_chunk_length = 3 * 24,
        training_length = 5 * 24,
        n_rnn_layers = 3,
        hidden_dim = 50,
        n_epochs = 100,
        dropout = 0.1,
    )
    
    rnn_model.fit([tars[tars_per][loc][type] for loc in locs], future_covariates=[covs[covs_per][loc][type] for loc in locs], verbose=True)
    for loc in locs:
        ys_rnn[loc] = rnn_model.predict(n_1hour, tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_rnn:
    y = extract_and_save(ys_rnn, "rnn_pca_1hour.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()

## CatBoost with optuna for hyperparamater optimization

Train and make predicitons

In [ ]:
import optuna
from darts.models import CatBoostModel
from darts.metrics import mae

tars_per = "1hour"
covs_per = "1hour"
type = "full"

ys_cat_optuna = {}

# Create a TimeSeries object
train_series, val_series = tars[tars_per]["A"][type].split_after(7*24)
future_covariates = covs[covs_per]["A"][type]

def objective(trial):
    trial_number = trial.number
    print(f"\nStarting Trial #{trial_number + 1}")
    
    # Define the search space for common CatBoost hyperparameters
    params = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0),
        # Add other hyperparameters as needed
    }

    # Create a CatBoost model with the suggested hyperparameters
    model = CatBoostModel(lags=5*24, lags_future_covariates=(24, 24), **params)

    # Train the model
    model.fit(train_series, future_covariates=future_covariates)

    # Make predictions on the validation set
    val_predictions = model.predict(len(val_series), future_covariates=future_covariates)

    # Evaluate the model using Mean Absolute Error (MAE)
    val_mae = mae(val_series, val_predictions)

    return val_mae

do_cat_optuna = False
if do_cat_optuna:

    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    
    # Get the best parameters and their corresponding mean absolute error (MAE)
    best_params = study.best_params
    best_mae = study.best_value
    
    print("Best Parameters:", best_params)
    print("Best Validation MAE:", best_mae)

    for loc in locs:
        cat = CatBoostModel(lags=5*24, lags_future_covariates=(24, 24), **best_params)
        cat.fit(tars[tars_per][loc][type], future_covariates=covs[covs_per][loc][type])
        ys_cat_optuna[loc] = cat.predict(n_1hour, future_covariates=covs[covs_per][loc][type])

In [ ]:
if do_cat_optuna:
    y = extract_and_save(ys_cat_optuna, "cat_optuna_1hour.csv", tars_per)
    if display_plots:
        y["prediction"].plot(lw=0.5)
        plt.show()